# Experiment Proper

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
import time

from ctmmodels.const import *

# Add more CTM models here as needed
from ctmmodels.delaythroughput import DTSimplexParentModel as NewModel
from ctmmodels.parentmodel import ParentModel as OldModel

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'dataframes/'

In [3]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

In [4]:
def run_model(demand, weights=(0.4, 0.4, 0.2), new_model=True, log_output=True):
    if new_model:
        model = NewModel(
            demand=demand,
            alpha=weights[0],
            beta=weights[1],
            gamma=weights[2],
            **parameters
        )
    else:
        model = OldModel(
            demand=demand,
            **parameters
        )

    model.generate()
    runtime = model.solve(log_output=log_output)
    dfx, dfy, dfg = model.return_solution()
    dfparams = model.return_parameters()
    obj_values = model.return_objective_value()
    return dfx, dfy, dfg, dfparams, obj_values, runtime, model

In [5]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Steps for Experiment Running

1. Generate the initial greentimes and objective values for perusal later in the discussion
2. Use the initial greentimes in running the LocalSim simulator (USE THE SAME PARAMETERS)
3. Record the animation
4. Obtain the results and preprocess for later

## Current Parameters

* **6-20 seconds** greentime range
* **30 time steps** or 60 seconds
* **5 runs** in LocalSim

## 1. Generate initial greentimes

In [6]:
simplex_points = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
]

demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

In [7]:
# Generate the new model greentimes

_df_tuples = []
vols = []
delqs = []

for d in demands:
    for pt in simplex_points:
        dfx, dfy, dfg, dfparams, objv, runtime, model = run_model(d, weights=pt, new_model=True, log_output=True)
        
        # Save the dataframes
        save_df(dfx, 'newextension/volumes/volumes_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        save_df(dfy, 'newextension/flows/flows_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        save_df(dfg, 'newextension/greentimes/greentimes_d{}_new_a{}_b{}_c{}'.format(d, pt[0], pt[1], pt[2]))
        
        # Get additional results
        vol = model.return_volume()
        delq = model.return_delay_equity()
        
        # Save the other results
        if isinstance(d, int):
            d_tuple = (d,d)
        else:
            d_tuple = d

        _df_tuples.append(
            (d_tuple[0], d_tuple[1], runtime, objv[0], objv[1], objv[2], pt[0], pt[1], pt[2])
        )
        
        vols.append(
            (d_tuple[0], d_tuple[1], vol[0], vol[1], vol[2], vol[3], pt[0], pt[1], pt[2])
        )
        
        delqs.append(
            (d_tuple[0], d_tuple[1], delq[0], delq[1], delq[2], delq[3], pt[0], pt[1], pt[2])
        )
    
    # Old model
    dfx, dfy, dfg, dfparams, objv, runtime, model = run_model(d, new_model=False, log_output=True)
        
    # Save the dataframes
    save_df(dfx, 'newextension/volumes/volumes_d{}_old'.format(d))
    save_df(dfy, 'newextension/flows/flows_d{}_old'.format(d))
    save_df(dfg, 'newextension/greentimes/greentimes_d{}_old'.format(d))
        
    # Get additional results
    vol = model.return_volume()
    delq = model.return_delay_equity()

    # Save the other results
    if isinstance(d, int):
        d_tuple = (d,d)
    else:
        d_tuple = d

    _df_tuples.append(
        (d_tuple[0], d_tuple[1], runtime, objv[0], objv[1], objv[2], 0, 0, 0)
    )
        
    vols.append(
        (d_tuple[0], d_tuple[1], vol[0], vol[1], vol[2], vol[3], 0, 0, 0)
    )

    delqs.append(
        (d_tuple[0], d_tuple[1], delq[0], delq[1], delq[2], delq[3], 0, 0, 0)
    )

Solving...
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 3212 rows and 340 columns.
MIP Presolve modified 704 coefficients.
Aggregator did 590 substitutions.
Reduced MIP has 3308 rows, 1890 columns, and 11464 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (14.25 ticks)
Found incumbent of value 1720.044950 after 0.04 sec. (25.28 ticks)
Probing time = 0.00 sec. (0.49 ticks)
Tried aggregator 1 time.
Reduced MIP has 3308 rows, 1890 columns, and 11464 nonzeros.
Reduced MIP has 270 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (8.01 ticks)
Probing time = 0.00 sec. (0.49 ticks)
Clique table members: 360.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.14 sec. (144.79 ticks)

        Nodes                                         Cuts/

  36343 18170      405.5094    84      423.5380      333.5325  2210063   21.25%
  37087 18641        cutoff            423.5380      334.1565  2256317   21.10%
  37859 19075      389.2661    92      423.5380      334.6608  2303102   20.98%
  38671 19622        cutoff            423.5380      335.2703  2355559   20.84%
  39435 20210      350.8951   138      423.5380      335.7591  2405113   20.73%
Elapsed time = 96.58 sec. (66510.12 ticks, tree = 353.31 MB, solutions = 13)
  40172 20550      384.7522   128      423.5380      336.3344  2435858   20.59%
  40865 21115      391.4659    79      423.5380      337.0705  2499157   20.42%
  41598 21253        cutoff            423.5380      337.8508  2514458   20.23%
  42379 21845      375.9319   106      423.5380      338.2633  2579443   20.13%
  43150 22356      407.2971    60      423.5380      338.7445  2626105   20.02%
  43892 22805      364.0480   129      423.5380      339.3906  2674367   19.87%
  44590 23187      383.2092    99      423.

 261392 88945        cutoff            418.2206      386.7564 17988554    7.52%
 265559 89350      414.2374    86      418.2206      387.2501 18242452    7.41%
 269704 89820      412.8152    62      418.2206      387.6225 18551118    7.32%
 273957 90206        cutoff            418.2206      388.0644 18846860    7.21%
 278218 90496        cutoff            418.2206      388.4412 19074694    7.12%
 282542 90809      418.1062    72      418.2206      388.8392 19359715    7.03%
 286932 91069        cutoff            418.2206      389.3337 19619193    6.91%
Elapsed time = 512.68 sec. (326958.50 ticks, tree = 467.32 MB, solutions = 18)
 291290 91244        cutoff            418.2206      389.7312 19877283    6.81%
 295696 91441        cutoff            418.2206      390.1009 20150695    6.72%
 300006 91603        cutoff            418.2206      390.5181 20490937    6.62%
 304452 91652        cutoff            418.2206      390.9221 20755128    6.53%
 308820 91725      399.1692    93      41

   3338   783     -335.3209   226     -297.3722     -324.1337   346845    9.00%
   3480    94     -299.2424   159     -297.3722     -324.1337   374802    9.00%
   3627   221        cutoff           -297.3722     -324.1337   420385    9.00%
   3878   363     -319.6556   196     -297.3722     -324.1337   467643    9.00%
   4128   542     -302.0478   103     -297.3722     -324.1337   525328    9.00%
   4373   701     -311.3943   158     -297.3722     -324.1337   579533    9.00%
Elapsed time = 25.18 sec. (18525.23 ticks, tree = 1.31 MB, solutions = 8)
   4607   862     -315.6906   177     -297.3722     -324.1337   643116    9.00%
   4876  1067     -301.9219   140     -297.3722     -324.1337   723523    9.00%
   5118  1164     -310.1187   147     -297.3722     -324.1337   770629    9.00%
   5389  1317     -301.7204   152     -297.3722     -324.0299   834329    8.96%
   5671  1443     -321.5933   196     -297.3722     -323.4040   890849    8.75%
   5958  1674     -312.2408   153     -297.372

  50574 12300     -302.0478   104     -297.3722     -304.6436  7867097    2.45%
  52471 12247        cutoff           -297.3722     -304.2730  8090807    2.32%
Elapsed time = 191.01 sec. (136056.30 ticks, tree = 149.71 MB, solutions = 8)
  54510 11980     -302.8564   144     -297.3722     -303.9181  8341581    2.20%
  56432 11790     -300.6178   139     -297.3722     -303.6341  8488069    2.11%
  58301 11652        cutoff           -297.3722     -303.2198  8623959    1.97%
  60406 11016     -299.2424    37     -297.3722     -302.8994  8921694    1.86%
  62174 10806     -301.6045    81     -297.3722     -302.6166  9039010    1.76%
  64236 10367     -299.7099   122     -297.3722     -302.2707  9226205    1.65%
  66552  9378        cutoff           -297.3722     -301.7328  9459273    1.47%
  68364  8675     -299.1090   120     -297.3722     -301.4547  9667197    1.37%
  70956  7206        cutoff           -297.3722     -300.6935  9921296    1.12%
  73110  6160        cutoff           -297

...

      0     0      117.6746   218       90.6392      Cuts: 22     1277   29.83%
      0     0      117.3942   216       90.6392      Cuts: 10     1346   29.52%
      0     0      117.3942   211       90.6392       Cuts: 5     1349   29.52%
*     0+    0                           93.0696      117.3942            26.14%
      0     2      117.3942   211       93.0696      116.3335     1349   25.00%
Elapsed time = 1.78 sec. (1423.42 ticks, tree = 0.01 MB, solutions = 4)
     36    25      104.1748   104       93.0696      116.3335     5636   25.00%
*   114+   51                           95.4177      116.3335            21.92%
    132    85      101.6592    80       95.4177      116.3335    16473   21.92%
    252   183      101.7891   105       95.4177      116.3335    27754   21.92%
*   281   198      integral     0       96.1392      116.3335    29161   21.01%
    365   237      104.9310   100       96.1392      115.6299    38229   20.27%
*   406+  239                           96.4177 

      0     0     2436.7184   200     2765.3509      Cuts: 12      442   11.88%
*     0+    0                         2671.1385     2436.7184             8.78%
      0     2     2436.7184   190     2671.1385     2448.8915      442    8.32%
Elapsed time = 1.65 sec. (1466.38 ticks, tree = 0.01 MB, solutions = 4)
     16    17     2623.5332    74     2671.1385     2476.6954     5578    7.28%
*    49+   28                         2661.8627     2477.6001             6.92%
     88    48     2593.1330    62     2661.8627     2477.6001    12304    6.92%
*   113+   65                         2660.7405     2477.6001             6.88%
*   130+   65                         2652.7358     2477.6001             6.60%
*   136    84      integral     0     2643.1050     2477.6001    22011    6.26%
*   165    96      integral     0     2631.9626     2477.6001    24356    5.86%
*   276+  124                         2620.4823     2497.4726             4.69%
*   285   146      integral     0     2619.5777 

*   155    95      integral     0    -5200.6329    -5265.1615    17739    1.24%
    182    60        cutoff          -5200.6329    -5265.1615    20828    1.24%
    379    28        cutoff          -5200.6329    -5220.8998    32581    0.39%

Cover cuts applied:  9
Implied bound cuts applied:  44
Mixed integer rounding cuts applied:  1
Gomory fractional cuts applied:  6

Root node processing (before b&c):
  Real time             =    1.13 sec. (969.99 ticks)
Parallel b&c, 8 threads:
  Real time             =    1.14 sec. (774.73 ticks)
  Sync time (average)   =    0.27 sec.
  Wait time (average)   =    0.01 sec.
                          ------------
Total (root+branch&cut) =    2.27 sec. (1744.72 ticks)
Done!
Time elapsed: 2.3748588562
Solving...
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 3252 rows and 400 columns.
MIP Presolve modified 652 coefficients.
Aggregator did 564 substitutions.
Reduced MIP has 3294 rows, 1856 columns, a


        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         2524.0484    -5435.9729           315.37%
      0     0    -1413.3241   115     2524.0484    -1413.3241        0  155.99%
*     0+    0                         -958.1722    -1413.3241            47.50%
      0     0    -1411.1066   110     -958.1722      Cuts: 12      141   46.88%
      0     0    -1410.1626   110     -958.1722       Cuts: 6      183   46.88%
      0     0    -1409.7724   110     -958.1722   Flowcuts: 2      222   46.88%
*     0+    0                        -1165.3322    -1407.3384            20.77%
*     0+    0                        -1274.4381    -1407.3384            10.43%
      0     2    -1409.7724   110    -1274.4381    -1389.3636      222    9.02%
Elapsed time = 1.41 sec. (1157.75 ticks, tree = 0.01 MB, solutions = 4)
     94    57        cutoff          -1274.4381    -1389.3636    10

*  1903   445      integral     0      113.1392      115.3342   174597    1.94%
   2526   284      114.1593   106      113.1392      114.2785   232157    1.01%
   2816   162      113.2179    61      113.1392      114.2785   255421    1.01%
   3042   119        cutoff            113.1392      113.4585   257706    0.28%

Cover cuts applied:  30
Implied bound cuts applied:  133

Root node processing (before b&c):
  Real time             =    1.25 sec. (1097.13 ticks)
Parallel b&c, 8 threads:
  Real time             =    9.76 sec. (7523.04 ticks)
  Sync time (average)   =    1.68 sec.
  Wait time (average)   =    0.02 sec.
                          ------------
Total (root+branch&cut) =   11.01 sec. (8620.18 ticks)
Done!
Time elapsed: 11.1175870895


In [11]:
df = pd.DataFrame(data=_df_tuples,columns=['demand_ns', 'demand_ew', 'runtime', 'delay', 'throughput', 'objective_value', 'alpha', 'beta', 'gamma'])
save_df(df, "newextension/results")

In [9]:
df_vols = pd.DataFrame(data=vols,columns=['demand_ns', 'demand_ew', 'volume_n', 'volume_e', 'volume_s', 'volume_w', 'alpha', 'beta', 'gamma'])
save_df(df_vols, "newextension/volume_equity")

In [10]:
df_delqs = pd.DataFrame(data=delqs,columns=['demand_ns', 'demand_ew', 'delay_n', 'delay_e', 'delay_s', 'delay_w', 'alpha', 'beta', 'gamma'])
save_df(df_delqs, "newextension/delay_equity")

## 2. Run the experiments

**Notes:**

* USE THE SAME PARAMETERS
* Label the results Excel sheets later